#Library

In [ ]:
!pip install bitstring

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from collections import Counter
from math import log, ceil
from bitstring import BitArray, Bits
import os
import heapq
import time

#HUFFMAN

## Function

In [ ]:
import os
import heapq
import time


class BinaryTree:
    def __init__(self, value, frequ):
        self.value = value
        self.frequ = frequ
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.frequ < other.frequ

    def __eq__(self, other):
        return self.frequ == other.frequ


class Huffmancode:
    def __init__(self, path):
        self.path = path
        self.__heap = []
        self.__code = {}
        self.__reversecode = {}

    def __frequency_from_text(self, text):
        frequ_dict = {}
        for char in text:
            if char not in frequ_dict:
                frequ_dict[char] = 0
            frequ_dict[char] += 1
        return frequ_dict

    def __Build_heap(self, frequency_dict):
        for key in frequency_dict:
            frequency = frequency_dict[key]
            binary_tree_node = BinaryTree(key, frequency)
            heapq.heappush(self.__heap, binary_tree_node)

    def __Build_Binary_Tree(self):
        while len(self.__heap) > 1:
            binary_tree_node_1 = heapq.heappop(self.__heap)
            binary_tree_node_2 = heapq.heappop(self.__heap)
            sum_of_freq = binary_tree_node_1.frequ + binary_tree_node_2.frequ
            newnode = BinaryTree(None, sum_of_freq)
            newnode.left = binary_tree_node_1
            newnode.right = binary_tree_node_2
            heapq.heappush(self.__heap, newnode)

    def __Build_Tree_Code_Helper(self, root, curr_bits):
        if root is None:
            return
        if root.value is not None:
            self.__code[root.value] = curr_bits
            self.__reversecode[curr_bits] = root.value
            return
        self.__Build_Tree_Code_Helper(root.left, curr_bits + '0')
        self.__Build_Tree_Code_Helper(root.right, curr_bits + '1')

    def __Build_Tree_Code(self):
        root = heapq.heappop(self.__heap)
        self.__Build_Tree_Code_Helper(root, '')

    def __Build_Encoded_Text(self, text):
        encoded_text = ''
        for char in text:
            encoded_text += self.__code[char]
        return encoded_text

    def __Build_Padded_Text(self, encoded_text):
        padding_value = 8 - (len(encoded_text) % 8)
        for i in range(padding_value):
            encoded_text += '0'
        padded_info = "{0:08b}".format(padding_value)
        padded_encoded_text = padded_info + encoded_text
        return padded_encoded_text

    def __Build_Byte_Array(self, padded_text):
        array = []
        for i in range(0, len(padded_text), 8):
            byte = padded_text[i:i + 8]
            array.append(int(byte, 2))
        return array

    def compression(self):
        filename, file_extension = os.path.splitext(self.path)
        output_path = 'encoded_huffman1.txt'
        with open(self.path, 'r') as file, open(output_path, 'wb') as output:
            text = file.read()
            text = text.rstrip()
            frequency_dict = self.__frequency_from_text(text)
            self.__Build_heap(frequency_dict)
            self.__Build_Binary_Tree()
            self.__Build_Tree_Code()
            encoded_text = self.__Build_Encoded_Text(text)
            padded_text = self.__Build_Padded_Text(encoded_text)
            bytes_array = self.__Build_Byte_Array(padded_text)
            final_bytes = bytes(bytes_array)
            output.write(final_bytes)
        print("Compressed")
        return output_path

    def __Remove_Padding(self, text):
        padded_info = text[:8]
        extra_padding = int(padded_info, 2)
        text = text[8:]
        padding_removed_text = text[:-1 * extra_padding]
        return padding_removed_text

    def __Decompress_Text(self, text):
        decoded_text = ''
        current_bits = ''
        for bit in text:
            current_bits += bit
            if current_bits in self.__reversecode:
                character = self.__reversecode[current_bits]
                decoded_text += character
                current_bits = ""
        return decoded_text

    def decompress(self, input_path):
        filename, file_extension = os.path.splitext(input_path)
        output_path = 'decoded_huffman.txt'
        with open(input_path, 'rb') as file, open(output_path, 'w') as output:
            bit_string = ''
            byte = file.read(1)
            while byte:
                byte = ord(byte)
                bits = bin(byte)[2:].rjust(8, '0')
                bit_string += bits
                byte = file.read(1)
            actual_text = self.__Remove_Padding(bit_string)
            decompressed_text = self.__Decompress_Text(actual_text)
            output.write(decompressed_text)
        print("Decompressed")
        return


path = "/content/drive/MyDrive/Năm 3/kì 2/tính toán đa phương tiện/đồ án mẫu/huffman + fano/test.txt.txt"
path = "5.txt"
h = Huffmancode(path)

t0 = time.time()
output_path = h.compression()
t1 = time.time()
total_1 = t1 - t0

t2 = time.time()
h.decompress(output_path)
t3 = time.time()
total_2 = t3 - t2



Compressed
Decompressed


## Result


In [ ]:
print("Input size:", os.stat(path).st_size, "(bytes).")
print("Output size:", os.stat(output_path).st_size, "(bytes).")
print("Decode size:", os.stat('decode_huffman.txt').st_size, "(bytes).")
r = os.stat(path).st_size/os.stat(output_path).st_size
print("Tỷ số nén:", round(r,5))
print("Tỷ lệ nén:", round(1/r*100,2), "%")
print("Hiệu suất nén:", 100-round(1/r*100,2), "%")
print("Thời gian nén:", round(total_1,5), "(giây).")
print("Thời gian giải nén:", round(total_2,5), "(giây).")

#LZW

In [ ]:
import numpy as np

In [ ]:
import shutil
from struct import *

## Save and Read Binary file

In [ ]:
import struct
def save_binary_file(data, file_path):
    with open(file_path, 'wb') as f:
        f.write(struct.pack(f'{len(data)}I', *data))


def read_binary_file(file_path):
    with open(file_path, 'rb') as f:
        # Read the binary data from the file
        binary_data = f.read()

        # Unpack the binary data into a list
        data_length = len(binary_data) // struct.calcsize('I')
        data = struct.unpack(f'{data_length}I', binary_data)

        return list(data)


## Text

Text file

In [ ]:
from collections import defaultdict

def encoding(file_path):
    with open(file_path, 'r') as f:
        s1 = f.read()
    table = defaultdict(int)
    for i in range(256):
        ch = chr(i)
        table[ch] = i

    p = s1[0]
    code = 256
    output_code = []
    for i in range(len(s1)):
        if i != len(s1) - 1:
            c = s1[i + 1]
        if p + c in table:
            p = p + c
        else:
            output_code.append(table[p])
            #print(p, table[p])
            table[p + c] = code
            code += 1
            p = c
        c = ""
    output_code.append(table[p])
    #print(p, table[p])

    return output_code

def decoding(op, file_path):
    table = defaultdict(str)
    for i in range(256):
        ch = chr(i)
        table[i] = ch
    old = op[0]
    s = table[old]
    c = s[0]
    count = 256
    with open(file_path, 'w') as f:
        f.write(s)
        for i in range(len(op) - 1):
            n = op[i + 1]
            if n not in table:
                s = table[old]
                s += c
            else:
                s = table[n]
            f.write(s)
            c = s[0]
            table[count] = table[old] + c
            count += 1
            old = n


In [ ]:
from collections import defaultdict

def decoding(op, file_path):
    # Khởi tạo từ điển ban đầu với các giá trị mặc định là chuỗi rỗng
    table = defaultdict(str)
    for i in range(256):
        ch = chr(i)
        table[i] = ch
    # Khởi tạo các biến ban đầu
    old = op[0]  # Giá trị đầu tiên trong op
    s = table[old]  # Giá trị tương ứng với old trong từ điển
    c = s[0]  # Ký tự đầu tiên của s
    count = 256  # output code cho những giá trị không có trong từ điển

    # Mở tệp tin để ghi dữ liệu giải nén
    with open(file_path, 'w') as f:
        # Ghi giá trị ban đầu của s vào tệp tin
        f.write(s)
        # Lặp qua các phần tử trong op (trừ phần tử đầu tiên)
        for i in range(len(op) - 1):
            n = op[i + 1]  # Giá trị hiện tại trong op

            # Nếu giá trị n không có trong từ điển
            if n not in table:
                s = table[old]
                s += c
            else:
                s = table[n]
            # Ghi giá trị s vào tệp tin
            f.write(s)

            # Cập nhật ký tự c
            c = s[0]
            # Thêm một cặp khóa-giá trị mới vào từ điển
            table[count] = table[old] + c
            count += 1
            # Cập nhật giá trị old
            old = n


In [ ]:
inputfile_path="E.txt"
t0 = time.time()
output_code=encoding(inputfile_path)
t1 = time.time()
total_1 = t1-t0
save_binary_file(output_code, 'output.bin')
t2 = time.time()
Decode_code = read_binary_file('output.bin')
decoding(Decode_code,'decode_lzw.txt')
t3 = time.time()
total_2 = t3-t2


In [ ]:
import os

input_file_size = os.path.getsize(inputfile_path)  # Kích thước của input file (trước khi mã hóa)
output_file_size = os.path.getsize('output.bin')  # Kích thước của output file (sau khi mã hóa)

compression_ratio = input_file_size / output_file_size
print("Input size:", input_file_size, "(bytes).")
print("Output size:", output_file_size, "(bytes).")
print("Decode size:", os.stat('decode_lzw.txt').st_size, "(bytes).")
r = compression_ratio
print("Tỷ số nén:", round(r,5))
print("Tỷ lệ nén:", round(1/r*100,2), "%")
print("Hiệu suất nén:", 100-round(1/r*100,2), "%")
print("Thời gian nén:", round(total_1,5), "(giây).")
print("Thời gian giải nén:", round(total_2,5), "(giây).")

Input size: 314 (bytes).
Output size: 244 (bytes).
Decode size: 310 (bytes).
Tỷ số nén: 1.28689
Tỷ lệ nén: 77.71 %
Hiệu suất nén: 22.290000000000006 %
Thời gian nén: 0.00384 (giây).
Thời gian giải nén: 0.00413 (giây).


String
Nhập vào chuỗi để nén

In [ ]:
def encoding(s1):
    print("Encoding")
    table = {}
    for i in range(256):
        ch = chr(i)
        table[ch] = i

    p = ""
    c = ""
    p += s1[0]
    code = 256
    output_code = []
    print("String\tOutput_Code\tAddition")
    for i in range(len(s1)):
        if i != len(s1) - 1:
            c += s1[i + 1]
        if p + c in table:
            p = p + c
        else:
            print(p, "\t", table[p], "\t\t", p + c, "\t", code)
            output_code.append(table[p])
            table[p + c] = code
            code += 1
            p = c
        c = ""
    print(p, "\t", table[p])
    output_code.append(table[p])
    return output_code


def decoding(op):
    print("\nDecoding")
    table = {}
    for i in range(256):
        ch = chr(i)
        table[i] = ch
    old = op[0]
    s = table[old]
    c = ""
    c += s[0]
    print(s, end="")
    count = 256
    for i in range(len(op) - 1):
        n = op[i + 1]
        if n not in table:
            s = table[old] + c
        else:
            s = table[n]
        print(s, end="")
        c = ""
        c += s[0]
        table[count] = table[old] + c
        count += 1
        old = n


s = "WYS*WYGWYS*WYSWYSG"
output_code = encoding(s)
print("Output Codes are:", output_code)
decoding(output_code)


Encoding
String	Output_Code	Addition
W 	 87 		 WY 	 256
Y 	 89 		 YS 	 257
S 	 83 		 S* 	 258
* 	 42 		 *W 	 259
WY 	 256 		 WYG 	 260
G 	 71 		 GW 	 261
WY 	 256 		 WYS 	 262
S* 	 258 		 S*W 	 263
WYS 	 262 		 WYSW 	 264
WYS 	 262 		 WYSG 	 265
G 	 71
Output Codes are: [87, 89, 83, 42, 256, 71, 256, 258, 262, 262, 71]

Decoding
WYS*WYGWYS*WYSWYSG

#SHANNON

##Function

In [ ]:
def load_file(input):
  with open(input) as f:
    content = f.readlines()
    content = "".join(content)
    if content:
        return content
    else:
        print("Read file failed please try again")
        return ""

hex2bin = dict('{:x} {:04b}'.format(x,x).split() for x in range(16))
def get_bincode(number, places=""):
    if number == 0.0:
      return "0"*places
    hx = float(number).hex()
    p = hx.index('p')
    bn = ''.join(hex2bin.get(char, char) for char in hx[2:p])
    bn = (bn.strip('0'))
    whole, flt = bn.split(".")
    if places == "":
      places = len(flt)+1
    num = int(hx[p+2:])
    whole = "0" * (num - 1) + whole
    output = whole + flt
    if len(output) < places:
      output += "0" * (places - len(output))
    return output[:places]

def generate(char, total, _dict):
  # calculate probability
  probability = [(x[0]/total) for x in char]

  # calculate codework length
  code_length = [(ceil(-log(x,2))) for x in probability]

  # cumulative probability
  cumulative_probability = 0.0
  for i in range(len(code_length)):
    bin_code = get_bincode(cumulative_probability, code_length[i])
    _dict[char[i][1]] = bin_code
    cumulative_probability += probability[i]
  return _dict

##Encode

In [ ]:
input = "5.txt"
output = "encoded_shannon.txt"
dict_name = "_dict_shannon.csv"

t0 = time.time()
# Read file
content = load_file(input)

# Count character
count = Counter(content)
total = sum(count.values())
char = [x[::-1] for x in list(count.items())]
char.sort(reverse=True)

# Create dictionary
_dict = dict()
generate(char,total,_dict)

# Encode
code = ''
for c in content:
  code += _dict[c]
code_bin = BitArray(bin = code)
max_bin = len(code)

# Save file
# output
with open(output, "wb") as f:
    code_bin.tofile(f)
# dict
keys = []
values = []
for key in _dict:
  keys.append(_dict[key])
  values.append(key)
encoded = {'keys': list(keys), 'values': list(values)}
df = pd.DataFrame(encoded)
df.to_csv(dict_name, index=False)
t1 = time.time()
total_1 = t1-t0

##Decode

In [ ]:
decode_file = "decoded_shannon.txt"

t2 = time.time()
# Read dictionary
df = pd.read_csv(dict_name,dtype=str)
keys = list(df['keys'])
values = list(df['values'])
_dict = dict(zip(keys,values))
with open(output) as f:
  data = Bits(f)


# Decode
string = ''
s = ''
for c in data.bin[:max_bin]:
  if s+c not in _dict:
    s+=c
  else:
    string+=_dict[s+c]
    s = ''

# Save file
file = open(decode_file, 'w')
file.write(string)
file.close()
t3 = time.time()
total_2 = t3-t2

##Result

In [ ]:
print("Input size:", os.stat(input).st_size, "(bytes).")
print("Output size:", os.stat(output).st_size, "(bytes).")
print("Decode size:", os.stat(decode_file).st_size, "(bytes).")
r = os.stat(input).st_size/os.stat(output).st_size
print("Tỷ số nén:", round(r,5))
print("Tỷ lệ nén:", round(1/r*100,2), "%")
print("Hiệu suất nén:", 100-round(1/r*100,2), "%")
print("Thời gian nén:", round(total_1,5), "(giây).")
print("Thời gian giải nén:", round(total_2,5), "(giây).")

Input size: 2167737 (bytes).
Output size: 1232335 (bytes).
Decode size: 2167737 (bytes).
Tỷ số nén: 1.75905
Tỷ lệ nén: 56.85 %
Hiệu suất nén: 43.15 %
Thời gian nén: 1.12055 (giây).
Thời gian giải nén: 3.69857 (giây).


#FANO

##Function

In [ ]:
def load_file(input):
  with open(input) as f:
    content = f.readlines()
    content = "".join(content)
    if content:
        return content
    else:
        print("Read file failed please try again")
        return ""

def generate(bin_code="", char=[], total=0, _dict={}):
    if len(char) == 1:
        _dict[char[0][1]] = bin_code
        return
    count = char[0][0]
    i = 1
    while True:
        left = count + char[i][0]
        if left*2 > total:
            break
        count += char[i][0]
        i+=1
    generate(bin_code+'0',char[:i],count,_dict)
    generate(bin_code+'1',char[i:],total-count,_dict)

##Encode

In [ ]:
input = "5.txt"
output = "encoded_fano.txt"
dict_name = "_dict_fano.csv"

t0 = time.time()
# Read file
content = load_file(input)

# Count character
count = Counter(content)
total = sum(count.values())
char = [x[::-1] for x in list(count.items())]
char.sort(reverse=True)

# Create dictionary
_dict = dict()
generate('',char,total,_dict)

# Encode
code = ''
for c in content:
  code += _dict[c]
code_bin = BitArray(bin = code)
max_bin = len(code)

# Save file
# output
with open(output, "wb") as f:
    code_bin.tofile(f)
# dict
keys = []
values = []
for key in _dict:
  keys.append(_dict[key])
  values.append(key)
encoded = {'keys': list(keys), 'values': list(values)}
df = pd.DataFrame(encoded)
df.to_csv(dict_name, index=False)
t1 = time.time()
total_1 = t1-t0

##Decode

In [ ]:
decode_file = "decoded_fano.txt"

t2 = time.time()
# Read dictionary
df = pd.read_csv(dict_name,dtype=str)
keys = list(df['keys'])
values = list(df['values'])
_dict = dict(zip(keys,values))
with open(output) as f:
  data = Bits(f)


# Decode
string = ''
s = ''
for c in data.bin[:max_bin]:
  if s+c not in _dict:
    s+=c
  else:
    string+=_dict[s+c]
    s = ''

# Save file
file = open(decode_file, 'w')
file.write(string)
file.close()
t3 = time.time()
total_2 = t3-t2

##Result

In [ ]:
print("Input size:", os.stat(input).st_size, "(bytes).")
print("Output size:", os.stat(output).st_size, "(bytes).")
print("Decode size:", os.stat(decode_file).st_size, "(bytes).")
r = os.stat(input).st_size/os.stat(output).st_size
print("Tỷ số nén:", round(r,5))
print("Tỷ lệ nén:", round(1/r*100,2), "%")
print("Hiệu suất nén:", 100-round(1/r*100,2), "%")
print("Thời gian nén:", round(total_1,5), "(giây).")
print("Thời gian giải nén:", round(total_2,5), "(giây).")

Input size: 2167737 (bytes).
Output size: 1180838 (bytes).
Decode size: 2167737 (bytes).
Tỷ số nén: 1.83576
Tỷ lệ nén: 54.47 %
Hiệu suất nén: 45.53 %
Thời gian nén: 1.1209 (giây).
Thời gian giải nén: 3.62304 (giây).
